This document is generated from an IPython Notebook session.
The content is based on a data mining lecture slide _"Association Rule Discovery: Apriori Algorithm"_ by Assoc. Prof. Kitsana Waiyamai, Ph.D., which is in turn based on Tan, Steinbach, Kumar slides.

Introduction
============

\*ahem, ahem\* first, please let me import some Python modules.

In [1]:
from sympy import *
from itertools import *
init_printing()

I also want to generate k-subsets from a given set.

In [2]:
def subsets(S, k):
  return [set(s)
    for s in combinations(S, k)]

In [3]:
subsets({1, 2, 3}, 2)

And sometimes I want to print a list, each on its own line:

In [4]:
def print_all(iterable):
  for item in iterable:
    print item

In [5]:
print_all([1, 2, 3])

1
2
3


Yes, it's working! Now, let's come back to __association rule mining__.

In association rule mining, we try to discover rules based on item sets.

Our input data is a list of _transactions_. Each transaction contains a set of items (called an _itemset_).
For example, this table represents the set of items bought by a customer in a single transaction:

| TID | Items |
| --- | ----- |
| 1 | Bread, Milk |
| 2 | Bread, Diaper, Beer, Eggs |
| 3 | Milk, Diaper, Beer, Coke |
| 4 | Bread, Milk, Diaper, Beer |
| 4 | Bread, Milk, Diaper, Coke |

In [6]:
T = [
 {'Bread', 'Milk'},
 {'Beer', 'Bread', 'Diaper', 'Eggs'},
 {'Berr', 'Coke', 'Diaper', 'Milk'},
 {'Beer', 'Bread', 'Diaper', 'Milk'},
 {'Bread', 'Coke', 'Diaper', 'Milk'},
]

We use them to find association rules, such as:

$$ \{\text{Milk}, \text{Bread}\} \Rightarrow \{\text{Eggs}, \text{Coke}\} $$

Which means that if a customer buys Milk and Bread, it is likely that they will buy Eggs and Coke as well.

Itemsets
========

An _itemset_ is simply a set of items, such as $\{ \text{Milk}, \text{Bread}, \text{Eggs} \}$.

The frequency of occurrence of an _itemset_ is called support count.

$$ \sigma(X) = \|\{ x \in T \:|\: X \subseteq x \}\| $$

In [7]:
def support_count(X):
  return S(sum(1 for x in T if X <= x))

In [8]:
support_count({'Milk', 'Bread', 'Diaper'})

Support is the _proportion_ of transactions that contain an itemset.

$$ s(X) = \frac{\sigma(X)}{\|\text{T}\|} $$

In [9]:
def support(X):
  return support_count(X) / len(T)

In [10]:
support({'Milk', 'Bread', 'Diaper'})

If the _support_ is higher than a given ratio, it is called a _frequent itemset_.

Rules
=====

A rule is in form of $X \Rightarrow Y$ where $X$ and $Y$ are itemsets. For example:

$$ \{ \text{Milk}, \text{Diaper} \} \Rightarrow \{ \text{Beer} \} $$

In [11]:
rule = ({'Milk', 'Diaper'}, {'Beer'})

The _support_ of the rule is the fraction of transactions that contain both $X$ and $Y$.

$$ s(X \Rightarrow Y) = s(X \cup Y) $$

In [12]:
def rule_support((x, y)):
  return support(x | y)

In [13]:
rule_support(rule)

The _confidence_ of the rule tells you how many transactions that contains $X$ also contains $Y$ (in form of proportion).

$$ c(X \Rightarrow Y) = \frac{\sigma(X \cup Y)}{\sigma(X)} \left(= \frac{s(X \Rightarrow Y)}{s(X)}\right) $$

In [14]:
def rule_confidence((x, y)):
  return (support_count(x | y)
        / support_count(x))

In [15]:
rule_confidence(rule)

Association Rule Mining
=======================

In association rule mining, we want to find all rules that has enough _support_ and _confidence_. In other words, we want to find $\{ X \Rightarrow Y \:|\: s(X \Rightarrow Y) > s_{min}, c(X \Rightarrow Y) > c_{min} \}$.

We are using 2-step approach:

- First, finding frequent itemsets with enough support.
    - For example, $\{A, B, C\}$
- Then, generate rules from these itemsets.
    - We can generate rules by finding binary partitions of a given itemset.
    - For example, from $\{A, B, C\}$, we can generate 6 rules:
    - Note that support of these rules are all the same.
    - We then select only the rules with enough confidence.
        - $\{A\} \Rightarrow \{C,B\}$
        - $\{C\} \Rightarrow \{A,B\}$
        - $\{B\} \Rightarrow \{A,C\}$
        - $\{A,C\} \Rightarrow \{B\}$
        - $\{A,B\} \Rightarrow \{C\}$
        - $\{C,B\} \Rightarrow \{A\}$

Let's forget about finding frequent itemsets for now; let's assume it hath been given unto us.

For clarity and brevity, I'll just use the dataset from the instructor's slides.

In [16]:
T = [
 {'A', 'B', 'E'},
 {'B', 'D'},
 {'B', 'C'},
 {'A', 'B', 'D'},
 {'A', 'C'},
 {'B', 'C'},
 {'A', 'C'},
 {'A', 'B', 'C', 'E'},
 {'A', 'B', 'C'},
]

cmin = 0.5

Given a Frequent Itemset
------------------------

We can use a naive algorithm:

In [17]:
# frequent itemset `l`
def find_rules(l):
  rules = []
  for n in range(1, len(l)):
    for c in subsets(l, n):
      rule = (set(c), l - set(c))
      if rule_confidence(rule) >= cmin:
        rules.append(rule)
  return rules

It just tries all "binary partitions" of the frequent itemset `l`, and only emits rules with enough confidence.

In [18]:
[(rule, rule_confidence(rule))
  for rule in find_rules({'A', 'B', 'E'})]

[(({'E'}, {'A', 'B'}), 1),
 (({'A', 'B'}, {'E'}), 1/2),
 (({'A', 'E'}, {'B'}), 1),
 (({'B', 'E'}, {'A'}), 1)]

Without a Frequent Itemset
--------------------------

Now, recall about having to find the frequent itemsets. What should we do?

For a large dataset, this is impractical. To find all the candidates, we must try all subsets of all items!
If we have $d$ items, the number of subsets become $2^d$.
See how fast it grows!
What can we do to help?

Apriori Principle
-----------------

You know, given that $A \subseteq B$, then $C \subseteq B$ as well if $C \subseteq B$.
It is also obvious that if $A \subseteq B$, $s(A) \geq s(B)$, since every item in $A$ also appear in $B$.

Apriori priciple says:
_"If an itemset is frequent, then all of its subsets must also be frequent."_

From that, we know that if an itemset is _infrequent_, all its supersets are also infrequent.

Again, let's use the example from the slides.

In [19]:
T = [
  {'A', 'C', 'D'},
  {'B', 'C', 'E'},
  {'A', 'B', 'C', 'E'},
  {'B', 'E'},
  {'A', 'B', 'C', 'E'}
]

smin = 0.4


### Starting Small

So here's our approach.

- First, we find all frequent itemsets of size 1 (called _frequent 1-itemsets_).
- Next, we "prune" itemsets whose support is too low.
- Then, we generate frequent _2_-itemsets from the remaining 1-itemsets.
- Again, we "prune" itemsets whose support is too low.
- Increase the size and repeat.


### Generating Frequent 1-itemsets

First, let's generate frequent 1-itemsets.
Before that, I will create a function to union multiple sets.

In [20]:
def union_all(sets):
  """Finds the union of given sets."""
  result = set()
  for c in sets:
    result = result | c
  return result

In [21]:
union_all([{1, 2}, {2, 3}])

In [22]:
union_all([])

Now, some code to find frequent 1-itemsets:

In [23]:
def frequent_1():
  items = union_all(T)
  return [{item}
    for item in items
      if support({item}) >= smin]

In [24]:
L1 = frequent_1()
L1

[{'A'}, {'C'}, {'B'}, {'E'}]

As you see, "D" is eliminated from the candidates.
That means any itemset with "D" in it will not be frequent enough.

### Expanding It

Next, we generate frequent 2-itemsets from 1-itemsets.
The easiest way to do it is to put these items together and select 2 items. First, we put them together:

In [25]:
union_all(L1)

{'A', 'B', 'C', 'E'}

In [26]:
C2 = subsets(_, 2)
print_all(C2)

set(['A', 'C'])
set(['A', 'B'])
set(['A', 'E'])
set(['C', 'B'])
set(['C', 'E'])
set(['B', 'E'])


These are the candidate itemsets. But maybe... not all of them are frequent enough.

Now, for each candidate $c$ in `C2`,
we must make sure at all of $c$'s 1-subset is in `L1`.
Why? If one of $c$'s subset (let's call it $s$) is not in `L1`,
it means that that $s$ has already been pruned, because $s$ is not frequent enough.
Since $c$ is a superset of $s$, $c$ will also not be frequent enough.

In [27]:
def good_candidate(c, P):
  for s in subsets(c, len(c) - 1):
    if s not in P: return false
  return True

In [28]:
F2 = [c for c in C2 if good_candidate(c, L1)]
print_all(F2)

set(['A', 'C'])
set(['A', 'B'])
set(['A', 'E'])
set(['C', 'B'])
set(['C', 'E'])
set(['B', 'E'])


Well, it seems that every candidate is a good one. Anyway, now we have the finalists!
For the final round, you might have guessed it:
We simply check the support to see if each item set is frequent enough!

In [29]:
L2 = [f for f in F2 if support(f) >= smin]
print_all(L2)

set(['A', 'C'])
set(['A', 'B'])
set(['A', 'E'])
set(['C', 'B'])
set(['C', 'E'])
set(['B', 'E'])


Again, all of them are frequent enough! So, now we have the 2-itemsets.

### Moving On

Now, let's generate `L3`.

In [30]:
def generate_candidates(P, k):
  return subsets(union_all(P), k)

In [31]:
C3 = generate_candidates(L2, 3)
print_all(C3)

set(['A', 'C', 'B'])
set(['A', 'C', 'E'])
set(['A', 'B', 'E'])
set(['C', 'B', 'E'])


In [32]:
F3 = [c for c in C3
         if good_candidate(c, L2)]
print_all(F3)

set(['A', 'C', 'B'])
set(['A', 'C', 'E'])
set(['A', 'B', 'E'])
set(['C', 'B', 'E'])


In [33]:
L3 = [f for f in F3
         if support(f) >= smin]
print_all(L3)

set(['A', 'C', 'B'])
set(['A', 'C', 'E'])
set(['A', 'B', 'E'])
set(['C', 'B', 'E'])


### Generalizing It

We can turn the above steps into this function:

In [34]:
def frequent_k(P, k):
  C = generate_candidates(P, k)
  F = [c for c in C if good_candidate(c, P)]
  return [f for f in F if support(f) >= smin]

We then use that function to generate `L4`.

In [35]:
L4 = frequent_k(L3, 4)
L4

[{'A', 'B', 'C', 'E'}]

Finally, generating `L5` will return no itemsets, which concludes the Apriori algorithm:

In [36]:
L5 = frequent_k(L4, 5)
L5

Putting It Together
-------------------

We take all the previous answers to find the frequent itemsets!

In [37]:
L1 + L2 + L3 + L4 + L5

[{'A'},
 {'C'},
 {'B'},
 {'E'},
 {'A', 'C'},
 {'A', 'B'},
 {'A', 'E'},
 {'B', 'C'},
 {'C', 'E'},
 {'B', 'E'},
 {'A', 'B', 'C'},
 {'A', 'C', 'E'},
 {'A', 'B', 'E'},
 {'B', 'C', 'E'},
 {'A', 'B', 'C', 'E'}]

Summing It Up
-------------

Finally, here's the apriori algorithm!

In [38]:
def apriori():
  result = []
  L = frequent_1()
  k = 1
  while len(L) > 0:
    result += L
    k += 1
    L = frequent_k(L, k)
  return result

In [39]:
L = apriori()
L

[{'A'},
 {'C'},
 {'B'},
 {'E'},
 {'A', 'C'},
 {'A', 'B'},
 {'A', 'E'},
 {'B', 'C'},
 {'C', 'E'},
 {'B', 'E'},
 {'A', 'B', 'C'},
 {'A', 'C', 'E'},
 {'A', 'B', 'E'},
 {'B', 'C', 'E'},
 {'A', 'B', 'C', 'E'}]

For each frequent itemsets, we generate rules from it.

In [40]:
cmin = 0.75

[rule
  for itemset in L
    for rule in find_rules(itemset)]

[({'A'}, {'C'}),
 ({'C'}, {'A'}),
 ({'C'}, {'B'}),
 ({'B'}, {'C'}),
 ({'C'}, {'E'}),
 ({'E'}, {'C'}),
 ({'B'}, {'E'}),
 ({'E'}, {'B'}),
 ({'A', 'B'}, {'C'}),
 ({'A', 'E'}, {'C'}),
 ({'A', 'B'}, {'E'}),
 ({'A', 'E'}, {'B'}),
 ({'C'}, {'B', 'E'}),
 ({'B'}, {'C', 'E'}),
 ({'E'}, {'B', 'C'}),
 ({'B', 'C'}, {'E'}),
 ({'C', 'E'}, {'B'}),
 ({'B', 'E'}, {'C'}),
 ({'A', 'B'}, {'C', 'E'}),
 ({'A', 'E'}, {'B', 'C'}),
 ({'A', 'B', 'C'}, {'E'}),
 ({'A', 'C', 'E'}, {'B'}),
 ({'A', 'B', 'E'}, {'C'})]

The Lift
--------

How can you be sure that there really is a correlation between the itemset $X$ and $Y$?

Take an example from the slides:

- 90% of customers buy coffee.
- 25% of customers buy tea.
- 20% of customers buy both.

After filling Venn diagram, here's our transactions:

In [41]:
T = (
  20 * [{'coffee', 'tea'}] +
  70 * [{'coffee'}] +
   5 * [{'tea'}] +
   5 * [set()]
)

Given $\{\text{coffee}, \text{tea}\}$ is a frequent itemset,
let's mine some rules!

In [42]:
rules = find_rules({'coffee', 'tea'})
rules

[({'tea'}, {'coffee'})]

Here, we mined the rule $\{\text{tea}\} \Rightarrow \{\text{coffee}\}$.
How confident we are?

In [43]:
rule = ({'tea'}, {'coffee'})
rule_confidence(rule)

We found that __80% of customers that buy tea also buys coffee__.
We're highly confident, at 80 percent!
But is 80% good?

Does it really mean that the customer buys coffee _because_ they buys tea?
To find out, let's remove the condition.
Let's see how many people buys coffee "no matter what:"

In [44]:
rule = (set(), {'coffee'})
rule_confidence(rule)

This is called the unconditional, _expected confidence_.
As you see, people who buy tea are actually less likely to buy coffee.
This is a _negative correlation_.

To quantify this correlation, we use a measure called "lift:"

$$ L(X \Rightarrow Y) = \frac{c(X \Rightarrow Y)}{c(\varnothing \Rightarrow Y)}$$

In [45]:
def rule_lift((x, y)):
  return (rule_confidence((x, y))
        / rule_confidence((set(), y)))

In [46]:
rule = ({'tea'}, {'coffee'})
rule_lift(rule)

The "lift" measure tells us the correlation of the rule.

- $L > 1 \Rightarrow$ positive correlation
- $L = 1 \Rightarrow$ independence
- $L < 1 \Rightarrow$ negative correlation

And this concludes this notebook regarding association rules and apriori algorithm.